**Import Library**

In [1]:
import pandas as pd
import numpy as np

In [2]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout #to reduce overfitting and increase model performance

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

**Read CSV file**

In [5]:
df = pd.read_csv('train.csv')
#df = pd.read_csv('train.csv',error_bad_lines=False,engine='python')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


**Drop NA**

In [7]:
df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
df = df.dropna()

**Get the Independent Features**

In [9]:
X = df.drop('label',axis=1)

In [10]:
X.shape

(18285, 4)

In [11]:
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


**Get the Dependent Features**

In [12]:
y = df['label']

In [13]:
y.shape

(18285,)

In [14]:
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

**Fix Vocabulary size**

In [15]:
voc_size = 5000 #hyperparameter tuning

**One Hot Representation**

In [16]:
df['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [17]:
messages = X.copy()

In [18]:
messages['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [19]:
messages.head(10)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [20]:
messages.reset_index(inplace=True)
messages.head(10)

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
6,7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
7,9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
8,10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
9,11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [21]:
###Dataset preprocessing
ps = PorterStemmer()

In [22]:
corpus =[]
for i in range(0,len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

In [23]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [24]:
onehot_repr = [one_hot(words,voc_size) for words in corpus]
onehot_repr

[[4842, 3966, 1617, 1839, 3646, 965, 4117, 2635, 1496, 4822],
 [4100, 192, 1324, 1517, 3334, 1237, 443],
 [2907, 381, 828, 1786],
 [3482, 796, 2638, 4510, 290, 2126],
 [4871, 3334, 1215, 4779, 3622, 1631, 3334, 1992, 1044, 4498],
 [2825,
  3207,
  693,
  1885,
  1529,
  1799,
  37,
  2506,
  4214,
  1796,
  525,
  2911,
  1016,
  725,
  443],
 [2353, 3589, 748, 3323, 3767, 4703, 4223, 257, 4426, 1133, 3611],
 [1600, 2756, 3077, 940, 3204, 3377, 1799, 1482, 4426, 1133, 3611],
 [1818, 119, 2148, 1349, 4816, 3907, 624, 4337, 1799, 2557],
 [3931, 965, 3240, 318, 2407, 3044, 4384, 2946],
 [1372, 3961, 1115, 319, 1711, 4273, 2419, 4402, 3868, 3310, 4663],
 [4510, 4870, 3646, 3907, 1799, 3204],
 [3893, 1867, 4053, 95, 2490, 1103, 3005, 3583, 3817],
 [804, 2242, 2067, 4018, 1723, 3821, 2360, 4426, 1133, 3611],
 [2167, 1551, 802, 384, 882, 4426, 1133, 3611],
 [836, 2039, 673, 3982, 487, 1626, 1066, 2533, 430, 670],
 [4498, 2022, 192],
 [1533, 1466, 4007, 3654, 1799, 3366, 3513, 443],
 [2721, 19

In [25]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [26]:
onehot_repr[1]

[4100, 192, 1324, 1517, 3334, 1237, 443]

**Embedding Representation : word to vec**

In [27]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2635 1496 4822]
 [   0    0    0 ... 3334 1237  443]
 [   0    0    0 ...  381  828 1786]
 ...
 [   0    0    0 ... 4426 1133 3611]
 [   0    0    0 ... 1390 3256 1415]
 [   0    0    0 ... 3287 3240 4256]]


In [28]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4842,
       3966, 1617, 1839, 3646,  965, 4117, 2635, 1496, 4822])

**Create Model**

In [29]:
embedding_vector_features = 40 #hyperparameter tuning #feature representation
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length)) #adding embedding layer
model.add(LSTM(100)) #LSTM with 100 neurons
model.add(Dense(1,activation='sigmoid')) #whenever o/p is binary, we use activation=sigmoid
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
len(embedded_docs),y.shape

(18285, (18285,))

In [31]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [32]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

**Model Training**

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X_final,y_final,test_size=0.33,random_state=42)
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 4s 15ms/step - loss: 0.3303 - accuracy: 0.8414 - val_loss: 0.2178 - val_accuracy: 0.9147
Epoch 2/10
192/192 [==============================] - 3s 14ms/step - loss: 0.1386 - accuracy: 0.9433 - val_loss: 0.1984 - val_accuracy: 0.9253
Epoch 3/10
192/192 [==============================] - 3s 16ms/step - loss: 0.0917 - accuracy: 0.9661 - val_loss: 0.2278 - val_accuracy: 0.9185
Epoch 4/10
192/192 [==============================] - 4s 18ms/step - loss: 0.0576 - accuracy: 0.9811 - val_loss: 0.2809 - val_accuracy: 0.9128
Epoch 5/10
192/192 [==============================] - 3s 17ms/step - loss: 0.0370 - accuracy: 0.9879 - val_loss: 0.2484 - val_accuracy: 0.9118
Epoch 6/10
192/192 [==============================] - 3s 17ms/step - loss: 0.0235 - accuracy: 0.9932 - val_loss: 0.3990 - val_accuracy: 0.9082
Epoch 7/10
192/192 [==============================] - 3s 18ms/step - loss: 0.0129 - accuracy: 0.9967 - val_loss: 0.4126 - val_accuracy: 0.9112

**Adding Dropout**

In [34]:
#embedding_vector_features = 40 #hyperparameter tuning #feature representation
#model = Sequential()
#model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length)) #adding embedding layer
#model.add(Dropout(0.3))
#model.add(LSTM(100)) #LSTM with 100 neurons
#model.add(Dropout(0.3))
#model.add(Dense(1,activation='sigmoid')) #whenever o/p is binary, we use activation=sigmoid
#model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

**Performance metrics and accuracy**

In [35]:
y_pred = model.predict(X_test)

189/189 [==============================] - 1s 3ms/step


In [40]:
y_pred = np.where(y_pred > 0.5, 1, 0) #use AUC ROC Curve to get the threshold, here we considered it as 0.5

In [37]:
confusion_matrix(y_test,y_pred)

array([[3093,  326],
       [ 193, 2423]], dtype=int64)

In [38]:
accuracy_score(y_test,y_pred)

0.9140016570008285

In [39]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.94      0.90      0.92      3419
           1       0.88      0.93      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.92      0.91      6035
weighted avg       0.92      0.91      0.91      6035

